In [ ]:
# basic settings
!pip install timm
!pip install adversarial-robustness-toolbox
!pip install timm foolbox


     |████████████████████████████████| 296kB 13.9MB/s 
     |████████████████████████████████| 972kB 16.6MB/s 
     |████████████████████████████████| 21.6MB 1.3MB/s 
     |████████████████████████████████| 235kB 47.0MB/s 
     |████████████████████████████████| 3.4MB 57.1MB/s 
     |████████████████████████████████| 747kB 65.5MB/s 
     |████████████████████████████████| 25.3MB 104.8MB/s 
  Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2
     |████████████████████████████████| 1.7MB 15.0MB/s 
     |████████████████████████████████| 163kB 21.8MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# drive.flush_and_unmount()
!pwd

Mounted at /content/drive
/content


In [ ]:
# For Vit importing
import timm

# For ViT examples
import urllib
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

# General + imagenet dataloader
from __future__ import print_function, division
import torchvision
from torchvision import datasets, models, transforms
import os
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time 
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

plt.ion()   # interactive mode

# ART lib
from art.attacks.evasion import FastGradientMethod, DeepFool
from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist

# Progress bar
# Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
# from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm

# foolbox
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import L2CarliniWagnerAttack, LinfPGD, LinfFastGradientAttack, L2PGD
import foolbox as fb


In [ ]:
#@title helper functions
def convert_1hot(data, num_classes=100):
  rows = len(data)
  cols = num_classes
  new_data = np.zeros((rows,cols))
  for idx, i in enumerate(data):
    new_data[idx,i] = 1
  return new_data


def count_parameters(model):
  num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
  print("The model has {} parameters".format(num_params))
  return num_params

def clear_pytorch_cache():
  torch.cuda.empty_cache()


def train_model(model, criterion, optimizer, scheduler, dataloaders, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


def show_cached_memory():
  print("Memory cached : {} MB".format(torch.cuda.memory_cached() / 1024**2))


# save image to folders
def save_adv_images(inputs, clipped_advs, labels, model, dirpath=""):

  dir = dirpath
  if len(dirpath) == "":
    timestr = time.strftime("%Y%m%d-%H%M%S")
    dirpath = os.path.join("/content/drive/MyDrive/598dataset", timestr)

  
  # try to create the dir
  os.makedirs(dir, exist_ok=True)

  # loop over images and save adv results
  clean_preds = model(inputs).argmax(axis=1)
  adv_preds = model(clipped_advs).argmax(axis=1)
  norm = ((inputs - clipped_advs)**2).sum(axis=(1, 2, 3))**0.5
  norm = norm.cpu().numpy().round(2)

  # remember dir count
  filenum_counter = {}

  for i in range(inputs.shape[0]):

    adv_img = clipped_advs[i].cpu().numpy()
    adv_img = adv_img.swapaxes(0, 1)
    adv_img = adv_img.swapaxes(1, 2)

    # get labels and predictions
    label = class_names[int(labels[i])]
    clean_pred = class_names[int(clean_preds[i])]
    adv_pred = class_names[int(adv_preds[i])]
    img_dir = os.path.join(dir, str(label))

    if img_dir not in filenum_counter:
      os.makedirs(img_dir, exist_ok=True)
      filenum_counter[img_dir] = len(os.listdir(img_dir)) + 1
    else:
      filenum_counter[img_dir] += 1

    # image name
    img_name = "img{}_clean{}_adv{}_L2diff{}.png".format(filenum_counter[img_dir], clean_pred, adv_pred, norm[i])
    img_path = os.path.join(img_dir, img_name)

    # save the image
    # print(adv_img.max(), adv_img.min(), adv_img.shape)
    plt.imsave(img_path, adv_img)

    # load the image and test

    

def torch_batch_to_img_array(batch):

  # render into numpy
  if str(type(batch)) == "<class 'list'>":
    batch = batch[0]
  
  # if 
  # batch = batch.cpu().numpy()


In [ ]:
def test_transfer(data_dir, fmodel, pin_memory=True):

  # clear the cache
  clear_pytorch_cache()

  # transforms
  data_transforms = {
    'val': transforms.Compose([
        transforms.ToTensor(),
    ]),
  }

  # datasets
  image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['val']}

  # image loaders
  dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4, pin_memory=pin_memory)
              for x in ['val']}


  # test accuracy 
  count = 0
  running_acc = 0

  show_cached_memory()
  with tqdm(dataloaders['val'], unit='batch') as tepoch:
    for inputs, labels in tepoch:
      inputs = inputs.to(device)
      labels = labels.to(device)
      running_acc += accuracy(fmodel, inputs, labels)
      count += 1

  print("Test Results on {}".format(data_dir))
  print(f"Accuracy:  {running_acc/count * 100:.1f} %")




In [ ]:
# load pre-trained vit
model = timm.create_model('vit_base_patch16_224', pretrained=True)

# For information about the implementation see here: https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/vision_transformer.py

for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.head.in_features
model.head = nn.Linear(num_ftrs, 16)
criterion = nn.CrossEntropyLoss()

# only parameters from the last layer is updated
optimizer_vit = optim.SGD(model.head.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_vit, step_size=7, gamma=0.1)

model_state_path = '/content/drive/MyDrive/598dataset/ViTbase_16_224_tiny_15classes.pth'
model.load_state_dict(torch.load(model_state_path))
model.eval()


In [ ]:
# initialize fmodel, test adv samples from ResNet50 to ViT
import os
model.eval()
preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
fmodel = PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)
NUM_CLASSES = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data_dirs = [
             '/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/Ensemble/FGSM/Epsilon0.001',
             '/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/Ensemble/FGSM/Epsilon0.005',
             '/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/Ensemble/FGSM/Epsilon0.01',
            '/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/Ensemble/FGSM/Epsilon0.1'
]

for data_dir in data_dirs:
  
  print(data_dir)
  test_transfer(data_dir, fmodel)
  print("\n")




/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/Ensemble/FGSM/Epsilon0.001
Memory cached : 382.0 MB


/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:375: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)



Test Results on /content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/Ensemble/FGSM/Epsilon0.001
Accuracy:  64.8 %


/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/Ensemble/FGSM/Epsilon0.005
Memory cached : 382.0 MB



Test Results on /content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/Ensemble/FGSM/Epsilon0.005
Accuracy:  46.1 %


/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/Ensemble/FGSM/Epsilon0.01
Memory cached : 382.0 MB



Test Results on /content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/Ensemble/FGSM/Epsilon0.01
Accuracy:  39.4 %


/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/Ensemble/FGSM/Epsilon0.1
Memory cached : 382.0 MB



Test Results on /content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/Ensemble/FGSM/Epsilon0.1
Accuracy:  25.3 %


